In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==============================
# 1️⃣ Load Data from CSV
# ==============================
csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"  # Update with your file path
df = pd.read_csv(csv_file_path)
print(df.columns)

# ==============================
# 2️⃣ Prepare Data (Separate Features & Labels)
# ==============================
X = df.drop(columns=['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
       'stayYears', 'assertLanguage', 'languagesSpoken'])  # Features
y = df['state']  # Target variable (predicting state)

# Encode labels (convert state names to numbers)
state_encoder = LabelEncoder()
y_encoded = state_encoder.fit_transform(y)

Index(['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
       'stayYears', 'assertLanguage', 'languagesSpoken', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37'],
      dtype='object')


In [ ]:
# prompt: show me classes of state encoder

# Get the unique classes (states) from the encoded labels
state_classes = list(state_encoder.classes_)

print("State Classes:")
state_classes

# You can access the numerical representation of each state using:
# state_encoder.transform(['Maharashtra', 'Karnataka'])

# You can get the original state name using:
# state_encoder.inverse_transform([0, 1])


State Classes:


['AndhraPradesh',
 'Bihar',
 'Chhattisgarh',
 'Goa',
 'Jharkhand',
 'Karnataka',
 'Maharashtra',
 'Rajasthan',
 'Telangana',
 'UttarPradesh',
 'WestBengal']

In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder

save_path = "/content/drive/My Drive/state_encoder.pkl"

# Save the LabelEncoder correctly
with open(save_path, "wb") as file:
    pickle.dump(state_encoder, file)  # ✅ Pass the encoder and the file object

# Load the LabelEncoder
with open(save_path, "rb") as file:
    loaded_encoder = pickle.load(file)

# Test the loaded encoder
print(loaded_encoder.classes_)


['AndhraPradesh' 'Bihar' 'Chhattisgarh' 'Goa' 'Jharkhand' 'Karnataka'
 'Maharashtra' 'Rajasthan' 'Telangana' 'UttarPradesh' 'WestBengal']


In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder

save_path = "/content/drive/My Drive/language_encoder.pkl"

# Save the LabelEncoder correctly
with open(save_path, "wb") as file:
    pickle.dump(language_encoder, file)  # ✅ Pass the encoder and the file object

# Load the LabelEncoder
with open(save_path, "rb") as file:
    lang_encoder = pickle.load(file)

# Test the loaded encoder
print(lang_encoder.classes_)


['BENGALI' 'BHATRI' 'CHATTISGARHI' 'GONDI' 'HALBI' 'HINDI' 'KANNADA'
 'KHORTHA' 'KONKANI' 'MAITHILI' 'MARATHI' 'MARWARI' 'RAJASTHANI' 'TELUGU'
 'URDU']


In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder

save_path = "/content/drive/My Drive/state_encoder.pkl"

# Save the LabelEncoder correctly
with open(save_path, "wb") as file:
    pickle.dump(state_encoder, file)  # ✅ Pass the encoder and the file object

# Load the LabelEncoder
with open(save_path, "rb") as file:
    loaded_encoder = pickle.load(file)

# Test the loaded encoder
print(loaded_encoder.classes_)


In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder

# Create and fit the label encoder
state_encoder = LabelEncoder()
labels = ["Hindi", "Telugu", "Marathi", "Konkani", "Bengali"]
state_encoder.fit(labels)

# Save the LabelEncoder correctly
with open("label_encoder.pkl", "wb") as file:
    pickle.dump(state_encoder, file)  # ✅ Pass the encoder and the file object

# Load the LabelEncoder
with open("label_encoder.pkl", "rb") as file:
    loaded_encoder = pickle.load(file)

# Test the loaded encoder
print(loaded_encoder.classes_)


In [ ]:
# ==============================
# 1️⃣ Load Data from CSV
# ==============================
csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"  # Update with your file path
df = pd.read_csv(csv_file_path)
print(df.columns)

# ==============================
# 2️⃣ Prepare Data (Separate Features & Labels)
# ==============================
X = df.drop(columns=['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
       'stayYears', 'assertLanguage', 'languagesSpoken'])  # Features
y = df['state']  # Target variable (predicting state)

# Encode labels (convert state names to numbers)
state_encoder = LabelEncoder()
y_encoded = state_encoder.fit_transform(y)

# Split data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

from imblearn.over_sampling import SMOTE

# Set n_neighbors to 1 since you have only 2 samples in the minority class
sm = SMOTE(k_neighbors=1)

# Apply SMOTE
X_sm_train, y_sm_train = sm.fit_resample(X_train, y_train)

print(f'''Shape of X before SMOTE: {X_train.shape}
Shape of X after SMOTE: {X_sm_train.shape}''')

# Normalize features
scaler = StandardScaler()
X_sm_train_scaled = scaler.fit_transform(X_sm_train)
X_test_scaled = scaler.transform(X_test)

Index(['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
       'stayYears', 'assertLanguage', 'languagesSpoken', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37'],
      dtype='object')
Shape of X before SMOTE: (59187, 38)
Shape of X after SMOTE: (152999, 38)


Code for training the model for predicting states


In [ ]:


# Reshape for LSTM (samples, time steps, features)
X_train_reshaped = X_sm_train_scaled.reshape(X_sm_train_scaled.shape[0], 1, X_sm_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# Convert labels to categorical
y_train_categorical = to_categorical(y_sm_train)
y_test_categorical = to_categorical(y_test)

# ==============================
# 3️⃣ Build BiLSTM Model
# ==============================
model = Sequential([
    Input(shape=(1, X_sm_train_scaled.shape[1])),  # Corrected shape to use shape[1]
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(state_encoder.classes_), activation='softmax')  # Number of classes in y
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,         # Number of epochs to wait before stopping if no improvement
    restore_best_weights=True  # Restore the model weights from the epoch with the best validation loss
)

# Train the model with early stopping
history = model.fit(
    X_train_reshaped, y_train_categorical,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_categorical),
    callbacks=[early_stopping]  # Add the EarlyStopping callback
)

# Save the trained model
model.save('/content/drive/MyDrive/my_trained_model.h5')

# ==============================
# 5️⃣ Evaluate Model
# ==============================
loss, accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"BiLSTM Model Accuracy: {accuracy:.4f}")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/my_trained_model.h5')


Code for predicting the states of new audios

In [ ]:

# ==============================
# 1️⃣ Load Data from CSV
# ==============================
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ==============================
# 1️⃣ Load the Saved Model
# ==============================
model = load_model('/content/drive/MyDrive/my_trained_model.h5')

# ==============================
# 2️⃣ Load and Preprocess the New Dataset
# ==============================
# Load the new dataset
new_csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/test_audio_final.csv"  # Update with your file path
new_df = pd.read_csv(new_csv_file_path)

# Ensure the new dataset has the same columns as the training data
# Drop the same columns as during training
X_new = new_df.drop(columns=['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
                            'stayYears', 'assertLanguage', 'languagesSpoken'])

# Normalize the new data using the same scaler as during training
# (Assuming you saved the scaler during training; otherwise, refit it)
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)  # Use the same scaler as during training

# Reshape for LSTM (samples, time steps, features)
X_new = X_new.reshape(X_new.shape[0], 1, X_new.shape[1])

# ==============================
# 3️⃣ Make Predictions
# ==============================
# Predict on the new data
predictions = model.predict(X_new)

# Convert predictions from probabilities to class labels
predicted_classes = np.argmax(predictions, axis=1)

# ==============================
# 4️⃣ Decode Predictions
# ==============================
# If the target variable was label-encoded during training, decode the predictions
# Load or recreate the LabelEncoder used during training
state_encoder = LabelEncoder()
state_encoder.fit(df['state'])  # Assuming `df` is the original training data

# Decode the predicted classes back to state names
predicted_states = state_encoder.inverse_transform(predicted_classes)

# Add the predicted states to the new dataset
new_df['predicted_state'] = predicted_states

# ==============================
# 5️⃣ Save or Display Results
# ==============================
# Save the new dataset with predictions to a CSV file
new_df.to_csv('new_dataset_with_predictions.csv', index=False)

# Display the first few rows with predictions
print(new_df.head())



2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 476ms/step
        id                                           filename  \
0  7563707  IISc_VaaniProject_M_BR_Araria_Nita36727_144641...   
1  7563721  IISc_VaaniProject_M_BR_Araria_71981266_1247070...   
2  7563709  IISc_VaaniProject_M_BR_Araria_Nita36727_144844...   
3  7563714  IISc_VaaniProject_M_BR_Araria_Nita36727_144641...   
4  7563713  IISc_VaaniProject_M_BR_Araria_Rahu87667_001310...   

                                            file_url  state  gender  pincode  \
0  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854325   
1  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar  female   854328   
2  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854325   
3  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854325   
4  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854328   

  district stayYears assertLanguage languagesSpoken  ...  feature_29  \
0   Araria       

In [ ]:
new_df

,id,filename,file_url,state,gender,pincode,district,stayYears,assertLanguage,languagesSpoken,...,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,predicted_state
0,7563707,IISc_VaaniProject_M_BR_Araria_Nita36727_144641...,https://vaani.iisc.ac.in//Audios/Araria/IISc_V...,Bihar,male,854325,Araria,21,HINDI,['Hindi'],...,24.213826,29.381877,14.743478,0.033548,0.011172,-0.025681,-0.073708,-0.039746,0.011990,Jharkhand
1,7563721,IISc_VaaniProject_M_BR_Araria_71981266_1247070...,https://vaani.iisc.ac.in//Audios/Araria/IISc_V...,Bihar,female,854328,Araria,39,HINDI,['Hindi'],...,21.145171,23.260481,31.949563,0.014118,-0.005542,0.056249,-0.050925,0.005657,0.001793,Chhattisgarh
2,7563709,IISc_VaaniProject_M_BR_Araria_Nita36727_144844...,https://vaani.iisc.ac.in//Audios/Araria/IISc_V...,Bihar,male,854325,Araria,21,HINDI,['Hindi'],...,25.709413,35.111273,14.684528,0.025488,0.024609,0.015288,-0.115441,-0.032343,0.001734,Maharashtra
3,7563714,IISc_VaaniProject_M_BR_Araria_Nita36727_144641...,https://vaani.iisc.ac.in//Audios/Araria/IISc_V...,Bihar,male,854325,Araria,21,HINDI,['Hindi'],...,24.790419,33.726518,14.984811,-0.015004,-0.089572,-0.149736,-0.045910,-0.001094,-0.050903,Telangana
4,7563713,IISc_VaaniProject_M_BR_Araria_Rahu87667_001310...,https://vaani.iisc.ac.in//Audios/Araria/IISc_V...,Bihar,male,854328,Araria,23,HINDI,['Hindi'],...,21.786797,26.326641,16.187632,0.004982,-0.054480,-0.004590,0.063494,-0.009028,-0.021561,Jharkhand
5,7563717,IISc_VaaniProject_M_BR_Araria_Rahu87667_001310...,https://vaani.iisc.ac.in//Audios/Araria/IISc_V...,Bihar,male,854328,Araria,23,HINDI,['Hindi'],...,19.801884,23.499042,15.239035,-0.014291,0.012650,-0.080028,0.104683,0.014374,-0.000841,Telangana
6,7584792,IISc_VaaniProject_M_RJ_Nagaur_Balv21710_072937...,https://vaani.iisc.ac.in//Audios/Nagaur/IISc_V...,Rajasthan,male,341023,Nagaur,20,MARWARI,"['Marwari', 'Marwadi']",...,19.329682,17.749703,15.808508,0.141660,0.000532,0.288419,-0.234678,0.096259,0.012698,Rajasthan
7,7584876,IISc_VaaniProject_M_RJ_Nagaur_Arju18568_035217...,https://vaani.iisc.ac.in//Audios/Nagaur/IISc_V...,Rajasthan,male,341024,Nagaur,27,RAJASTHANI,"['Rajasthani', 'Hindi']",...,17.555888,15.870106,14.698688,0.049085,-0.039405,0.059207,-0.173001,0.038262,0.030731,WestBengal
8,7584805,IISc_VaaniProject_M_RJ_Nagaur_Balv21710_072715...,https://vaani.iisc.ac.in//Audios/Nagaur/IISc_V...,Rajasthan,male,341023,Nagaur,20,MARWARI,"['Marwari', 'Marwadi']",...,20.373575,23.976266,20.213988,0.158566,0.113788,0.307405,-0.165980,0.105910,-0.032990,Rajasthan
9,7584829,IISc_VaaniProject_M_RJ_Nagaur_Vije16053_111444...,https://vaani.iisc.ac.in//Audios/Nagaur/IISc_V...,Rajasthan,male,341024,Nagaur,34,MARWARI,"['Marwari', 'Marwadi', 'Hindi']",...,21.599105,22.877773,28.405642,0.007638,0.035126,-0.009001,0.047112,0.025167,0.013869,Rajasthan


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accur=accuracy_score(new_df['state'],new_df['predicted_state'])
print(accur)
df['languagesSpoken'].value_counts()


0.35555555555555557


,count
languagesSpoken,
['Hindi'],30688
['Telugu'],9517
['Rajasthani'],6244
['Kannada'],4664
['Marathi'],3669
...,...
"['Telugu', 'Tamil', 'English']",2
['Gondi'],2
"['Hindi', 'Marathi']",1


In [ ]:
# ==============================
# 1️⃣ Load Data from CSV
# ==============================
csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_file_path)
print(df.columns)

# ==============================
# 2️⃣ Prepare Data (Separate Features & Labels)
# ==============================
X = df.drop(columns=['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                   'stayYears', 'assertLanguage', 'languagesSpoken'])  # Features
y = df['assertLanguage']  # Target variable (predicting language)

# Encode categorical features and labels
state_encoder = LabelEncoder()
language_encoder = LabelEncoder()

# Encode state column if it exists
if 'state' in X.columns:
    X['state'] = state_encoder.fit_transform(X['state'])

y = language_encoder.fit_transform(y)

Index(['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
       'stayYears', 'assertLanguage', 'languagesSpoken', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37'],
      dtype='object')


In [ ]:
# prompt: classes of language encoder

# Get the unique classes (languages) from the encoded labels
language_classes = list(language_encoder.classes_)

print("Language Classes:")
language_classes


Language Classes:


['BENGALI',
 'BHATRI',
 'CHATTISGARHI',
 'GONDI',
 'HALBI',
 'HINDI',
 'KANNADA',
 'KHORTHA',
 'KONKANI',
 'MAITHILI',
 'MARATHI',
 'MARWARI',
 'RAJASTHANI',
 'TELUGU',
 'URDU']

In [ ]:

# You can access the numerical representation of each language using:
# language_encoder.transform(['Hindi', 'English'])

# You can get the original language name using:
# language_encoder.inverse_transform([0, 1])

# ==============================
# 3️⃣ Build BiLSTM Model for Language Prediction
# ==============================

# Split data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Set n_neighbors to 1 since you have only 2 samples in the minority class
sm = SMOTE(k_neighbors=1)

# Apply SMOTE
X_sm_train, y_sm_train = sm.fit_resample(X_train, y_train)

print(f'''Shape of X before SMOTE: {X_train.shape}
Shape of X after SMOTE: {X_sm_train.shape}''')

# Normalize features
scaler = StandardScaler()
X_sm_train_scaled = scaler.fit_transform(X_sm_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for LSTM (samples, time steps, features)
X_train_reshaped = X_sm_train_scaled.reshape(X_sm_train_scaled.shape[0], 1, X_sm_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# Convert labels to categorical
y_train_categorical = to_categorical(y_sm_train)
y_test_categorical = to_categorical(y_test)

# Build BiLSTM model for language prediction
model_language = Sequential([
    Input(shape=(1, X_sm_train_scaled.shape[1])),
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(language_encoder.classes_), activation='softmax')
])

# Compile the model
model_language.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,         # Number of epochs to wait before stopping if no improvement
    restore_best_weights=True  # Restore the model weights from the epoch with the best validation loss
)

# Train the model with early stopping
history = model_language.fit(
    X_train_reshaped, y_train_categorical,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_categorical),
    callbacks=[early_stopping]
)


# Save the trained model
model_language.save('/content/drive/MyDrive/language_model.h5')


Language Classes:


NameError: name 'SMOTE' is not defined

Code for training model for predicting languages based on predicted state

In [ ]:


# ==============================
# 3️⃣ Train-Test Split
# ==============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# ==============================
# 4️⃣ Handle Class Imbalance (SMOTE on training data only)

# ==============================
from imblearn.over_sampling import RandomOverSampler
sm = RandomOverSampler()
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

print(f'''Shape before SMOTE: {X_train.shape}
Shape after SMOTE: {X_train_sm.shape}''')

# ==============================
# 5️⃣ Feature Scaling
# ==============================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_sm)
X_test_scaled = scaler.transform(X_test)  # Use same scaler on test data

# ==============================
# 6️⃣ Reshape for LSTM and One-Hot Encoding
# ==============================
# Reshape for LSTM (samples, time steps, features)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# One-hot encode the target variables
y_train_categorical = to_categorical(y_train_sm)
y_test_categorical = to_categorical(y_test)

# ==============================
# 7️⃣ Build BiLSTM Model
# ==============================
model = Sequential([
    Input(shape=(1, X_train_scaled.shape[1])),  # Input shape based on features
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(language_encoder.classes_), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# ==============================
# 8️⃣ Train Model
# ==============================
history = model.fit(
    X_train_reshaped, y_train_categorical,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_categorical),
    callbacks=[early_stopping]
)

# ==============================
# 9️⃣ Save and Evaluate Model
# ==============================
model.save('/content/drive/MyDrive/my_trained_model_for_language.h5')

loss, accuracy = model.evaluate(X_test_reshaped, y_test_categorical)
print(f"BiLSTM Model Accuracy: {accuracy:.4f}")
print(f"Number of classes: {len(language_encoder.classes_)}")

Index(['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
       'stayYears', 'assertLanguage', 'languagesSpoken', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37'],
      dtype='object')
Shape before SMOTE: (44390, 39)
Shape after SMOTE: (291570, 39)
Epoch 1/100
9112/9112 ━━━━━━━━━━━━━━━━━━━━ 122s 13ms/step - accuracy: 0.7926 - loss: 0.5860 - val_accuracy: 0.8783 - val_loss: 0.3346
Epoch 2/100
9112/9112 ━━━━━━━━━━━━━━━━━━━━ 118s 13ms/step - accur

925/925 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9473 - loss: 0.1591
BiLSTM Model Accuracy: 0.9468
Number of classes: 15


Code for predicting the language on test audios as well.


In [ ]:

# ==============================
# 1️⃣ Load Data from CSV
# ==============================
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ==============================
# 1️⃣ Load the Saved Model
# ==============================
model = load_model('/content/drive/MyDrive/my_trained_model_for_language.h5')

# ==============================
# 2️⃣ Load and Preprocess the New Dataset
# ==============================
# Load the new dataset
new_df

# Ensure the new dataset has the same columns as the training data
# Drop the same columns as during training
X_new = new_df.drop(columns=['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
                            'stayYears', 'assertLanguage', 'languagesSpoken'])
X_new['state']=state_encoder.transform(X_new['predicted_state'])

X_new=X_new.drop(columns=['predicted_state'])

X_new=X_new[['state', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13',
       'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18',
       'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23',
       'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28',
       'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33',
       'feature_34', 'feature_35', 'feature_36', 'feature_37']]
# Normalize the new data using the same scaler as during training
# (Assuming you saved the scaler during training; otherwise, refit it)
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)  # Use the same scaler as during training

# Reshape for LSTM (samples, time steps, features)
X_new = X_new.reshape(X_new.shape[0], 1, X_new.shape[1])

# ==============================
# 3️⃣ Make Predictions
# ==============================
# Predict on the new data
predictions = model.predict(X_new)

# Convert predictions from probabilities to class labels
predicted_classes = np.argmax(predictions, axis=1)

# ==============================
# 4️⃣ Decode Predictions
# ==============================


# Decode the predicted classes back to state names
predicted_language = language_encoder.inverse_transform(predicted_classes)

# Add the predicted states to the new dataset
new_df['predicted_language'] = predicted_language

# ==============================
# 5️⃣ Save or Display Results
# ==============================
# Save the new dataset with predictions to a CSV file
new_df.to_csv('new_dataset_with_predictions.csv', index=False)

# Display the first few rows with predictions
print(new_df.head())



2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 449ms/step
        id                                           filename  \
0  7563707  IISc_VaaniProject_M_BR_Araria_Nita36727_144641...   
1  7563721  IISc_VaaniProject_M_BR_Araria_71981266_1247070...   
2  7563709  IISc_VaaniProject_M_BR_Araria_Nita36727_144844...   
3  7563714  IISc_VaaniProject_M_BR_Araria_Nita36727_144641...   
4  7563713  IISc_VaaniProject_M_BR_Araria_Rahu87667_001310...   

                                            file_url  state  gender  pincode  \
0  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854325   
1  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar  female   854328   
2  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854325   
3  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854325   
4  https://vaani.iisc.ac.in//Audios/Araria/IISc_V...  Bihar    male   854328   

  district stayYears assertLanguage languagesSpoken  ...  feature_30  \
0   Araria       

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accur1=accuracy_score(new_df['assertLanguage'],new_df['predicted_language'])
print(accur1)


0.4222222222222222


### MHA (Sakshi) (RUN IN JUPYTER NOTEBOOK)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, MultiHeadAttention, LayerNormalization, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ==============================
# 1️⃣ Load Training Data
# ==============================
csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_file_path)
print("✅ Loaded training data!")

# ==============================
# 2️⃣ Preprocess Data
# ==============================
X = df.drop(columns=['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                     'stayYears', 'assertLanguage', 'languagesSpoken', 'state'], errors='ignore')
y = df['assertLanguage']

# Encode labels
language_encoder = LabelEncoder()
y = language_encoder.fit_transform(y)

# Save Label Encoder
joblib.dump(language_encoder,"/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/language_encoder.pkl")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Save training feature names
train_feature_names = list(X_train.columns)
joblib.dump(train_feature_names, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/train_feature_names.pkl")

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save Scaler
joblib.dump(scaler, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/scaler.pkl")

# Reshape for Multi-Head Attention
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# One-hot encode labels
y_train_categorical = to_categorical(y_train, num_classes=len(language_encoder.classes_))
y_test_categorical = to_categorical(y_test, num_classes=len(language_encoder.classes_))

print("✅ Preprocessing done!")

# ==============================
# 3️⃣ Define Multi-Head Attention Model (Functional API)
# ==============================
input_layer = Input(shape=(1, X_train.shape[1]))

# Multi-Head Attention Layer
attention_output = MultiHeadAttention(num_heads=4, key_dim=32)(input_layer, input_layer)
attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

# Flatten and Fully Connected Layers
x = Flatten()(attention_output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(len(language_encoder.classes_), activation='softmax')(x)

# Create Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ==============================
# 4️⃣ Train the Model
# ==============================
history = model.fit(
    X_train_reshaped, y_train_categorical,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_categorical),
    callbacks=[early_stopping]
)

# ==============================
# 5️⃣ Save Model
# ==============================
model.save("/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/final colab.h5")
print("✅ Model training complete and saved!")


✅ Loaded training data!
✅ Preprocessing done!
Epoch 1/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.4755 - loss: 1.6885 - val_accuracy: 0.5927 - val_loss: 1.2501
Epoch 2/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5798 - loss: 1.3133 - val_accuracy: 0.6183 - val_loss: 1.1609
Epoch 3/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6017 - loss: 1.2295 - val_accuracy: 0.6498 - val_loss: 1.0907
Epoch 4/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6149 - loss: 1.1936 - val_accuracy: 0.6493 - val_loss: 1.0601
Epoch 5/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6276 - loss: 1.1534 - val_accuracy: 0.6641 - val_loss: 1.0389
Epoch 6/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.6342 - loss: 1.1296 - val_accuracy: 0.6577 - val_loss: 1.0214
Epoch 7/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.6339 - loss: 1.1170 - val_accuracy: 0.6776 - val_loss: 0.9915
Epoch 8/100
1388/1388 ━━━━━━━━━━━━

✅ Model training complete and saved!


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# ==============================
# 1️⃣ Load Saved Model & Encoders
# ==============================
model_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/final colab.h5"
encoder_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/language_encoder.pkl"
scaler_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/scaler.pkl"
feature_names_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/train_feature_names.pkl"

# Load Model
model = tf.keras.models.load_model(model_path)
print("✅ Model loaded successfully!")

# Load Label Encoder
if os.path.exists(encoder_path):
    language_encoder = joblib.load(encoder_path)
else:
    raise FileNotFoundError("🚨 Label Encoder file not found. Run training script first.")

# Load Scaler
if os.path.exists(scaler_path):
    scaler = joblib.load(scaler_path)
else:
    raise FileNotFoundError("🚨 Scaler file not found. Run training script first.")

# Load Training Feature Names
if os.path.exists(feature_names_path):
    train_feature_names = joblib.load(feature_names_path)
else:
    raise FileNotFoundError("🚨 Training feature names file not found. Run training script first.")

# ==============================
# 2️⃣ Load and Process Test Data
# ==============================
test_csv_path ="/content/drive/MyDrive/IE3 Lab/Sakshi PC/test_audio_final.csv"
df_test = pd.read_csv(test_csv_path)

# Extract actual labels before dropping columns
actual_languages = df_test['assertLanguage'].tolist()  # Store actual languages

# Drop unnecessary columns
drop_columns = ['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                'stayYears', 'assertLanguage', 'languagesSpoken', 'state']
df_test = df_test.drop(columns=[col for col in drop_columns if col in df_test.columns])

# Ensure test data has same features as training
for feature in train_feature_names:
    if feature not in df_test.columns:
        df_test[feature] = 0  # Add missing feature with default value

# Reorder columns to match training
df_test = df_test[train_feature_names]

# ==============================
# 3️⃣ Scale and Reshape Test Data
# ==============================
X_test_new = scaler.transform(df_test)  # Scale using the same scaler
X_test_reshaped_new = X_test_new.reshape(X_test_new.shape[0], 1, X_test_new.shape[1])

# ==============================
# 4️⃣ Make Predictions
# ==============================
predictions = model.predict(X_test_reshaped_new)
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_labels = language_encoder.inverse_transform(predicted_class_indices)

# ==============================
# 5️⃣ Print Predictions vs Actual Labels
# ==============================
print("\n🎯 Model Predictions vs Actual Labels:")
print(f"{'Sample':<10}{'Actual':<20}{'Predicted':<20}")

for i, (actual, predicted) in enumerate(zip(actual_languages, predicted_labels)):
    print(f"{i+1:<10}{actual:<20}{predicted:<20}")

# ==============================
# 6️⃣ Compute Accuracy
# ==============================
y_test_encoded = language_encoder.transform(actual_languages)
correct_predictions = sum(np.array(predicted_labels) == np.array(actual_languages))
accuracy = correct_predictions / len(actual_languages)

print(f"\n✅ Final Test Accuracy: {accuracy:.4f}")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/final colab.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Hybrid Language model (BiLSTM +MHA)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Dropout, Input, MultiHeadAttention, LayerNormalization, Flatten, LSTM, Bidirectional
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ==============================
# 1️⃣ Load Training Data
# ==============================
csv_file_path ="/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_file_path)
print("✅ Loaded training data!")

# ==============================
# 2️⃣ Preprocess Data
# ==============================
X = df.drop(columns=['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                     'stayYears', 'assertLanguage', 'languagesSpoken', 'state'], errors='ignore')
y = df['assertLanguage']

# Encode labels
language_encoder = LabelEncoder()
y = language_encoder.fit_transform(y)

# Save Label Encoder
joblib.dump(language_encoder, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/language_encoder.pkl")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Save training feature names
train_feature_names = list(X_train.columns)
joblib.dump(train_feature_names,  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/train_feature_names.pkl")

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save Scaler
joblib.dump(scaler,  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/scaler.pkl")

# Reshape for Multi-Head Attention
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# One-hot encode labels
y_train_categorical = to_categorical(y_train, num_classes=len(language_encoder.classes_))
y_test_categorical = to_categorical(y_test, num_classes=len(language_encoder.classes_))

print("✅ Preprocessing done!")

# ==============================
# 3️⃣ Define Hybrid Multi-Head Attention + BiLSTM Model
# ==============================
input_layer = Input(shape=(1, X_train.shape[1]))

# Multi-Head Attention Layer
attention_output = MultiHeadAttention(num_heads=4, key_dim=32)(input_layer, input_layer)
attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

# BiLSTM Layers for Sequential Learning
bilstm_output = Bidirectional(LSTM(128, return_sequences=True))(attention_output)
bilstm_output = Dropout(0.3)(bilstm_output)
bilstm_output = Bidirectional(LSTM(64, return_sequences=False))(bilstm_output)
bilstm_output = Dropout(0.3)(bilstm_output)

# Fully Connected Layers
x = Dense(64, activation='relu')(bilstm_output)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(len(language_encoder.classes_), activation='softmax')(x)

# Create Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ==============================
# 4️⃣ Train the Model
# ==============================
history = model.fit(
    X_train_reshaped, y_train_categorical,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_categorical),
    callbacks=[early_stopping]
)

# ==============================
# 5️⃣ Save Model
# ==============================
model.save( "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final code.h5")
print("✅ Hybrid Model training complete and saved!")


✅ Loaded training data!
✅ Preprocessing done!
Epoch 1/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.4775 - loss: 1.7455 - val_accuracy: 0.6035 - val_loss: 1.2304
Epoch 2/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.6005 - loss: 1.2564 - val_accuracy: 0.6621 - val_loss: 1.0396
Epoch 3/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - accuracy: 0.6497 - loss: 1.0969 - val_accuracy: 0.7015 - val_loss: 0.9151
Epoch 4/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.6884 - loss: 0.9718 - val_accuracy: 0.7267 - val_loss: 0.8367
Epoch 5/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.7105 - loss: 0.9069 - val_accuracy: 0.7480 - val_loss: 0.7763
Epoch 6/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 42s 16ms/step - accuracy: 0.7320 - loss: 0.8342 - val_accuracy: 0.7659 - val_loss: 0.7256
Epoch 7/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.7432 - loss: 0.7986 - val_accuracy: 0.7756 - val_loss: 0.6928
Epoch 8/100
1388/1388 ━

✅ Hybrid Model training complete and saved!


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# ==============================
# 1️⃣ Load Saved Model & Encoders
# ==============================
model_path =  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final code.h5"
encoder_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/language_encoder.pkl"
scaler_path =  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/scaler.pkl"
feature_names_path =  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/train_feature_names.pkl"

# Load Model
model = tf.keras.models.load_model(model_path)
print("✅ Model loaded successfully!")

# Load Label Encoder
if os.path.exists(encoder_path):
    language_encoder = joblib.load(encoder_path)
else:
    raise FileNotFoundError("🚨 Label Encoder file not found. Run training script first.")

# Load Scaler
if os.path.exists(scaler_path):
    scaler = joblib.load(scaler_path)
else:
    raise FileNotFoundError("🚨 Scaler file not found. Run training script first.")

# Load Training Feature Names
if os.path.exists(feature_names_path):
    train_feature_names = joblib.load(feature_names_path)
else:
    raise FileNotFoundError("🚨 Training feature names file not found. Run training script first.")

# ==============================
# 2️⃣ Load and Process Test Data
# ==============================
test_csv_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/test_audio_final.csv"
df_test = pd.read_csv(test_csv_path)

# Extract actual labels before dropping columns
actual_languages = df_test['assertLanguage'].tolist()  # Store actual languages

# Drop unnecessary columns
drop_columns = ['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                'stayYears', 'assertLanguage', 'languagesSpoken', 'state']
df_test = df_test.drop(columns=[col for col in drop_columns if col in df_test.columns])

# Ensure test data has same features as training
for feature in train_feature_names:
    if feature not in df_test.columns:
        df_test[feature] = 0  # Add missing feature with default value

# Reorder columns to match training
df_test = df_test[train_feature_names]

# ==============================
# 3️⃣ Scale and Reshape Test Data
# ==============================
X_test_new = scaler.transform(df_test)  # Scale using the same scaler
X_test_reshaped_new = X_test_new.reshape(X_test_new.shape[0], 1, X_test_new.shape[1])

# ==============================
# 4️⃣ Make Predictions
# ==============================
predictions = model.predict(X_test_reshaped_new)
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_labels = language_encoder.inverse_transform(predicted_class_indices)

# ==============================
# 5️⃣ Print Predictions vs Actual Labels
# ==============================
print("\n🎯 Model Predictions vs Actual Labels:")
print(f"{'Sample':<10}{'Actual':<20}{'Predicted':<20}")

for i, (actual, predicted) in enumerate(zip(actual_languages, predicted_labels)):
    print(f"{i+1:<10}{actual:<20}{predicted:<20}")

# ==============================
# 6️⃣ Compute Accuracy
# ==============================
y_test_encoded = language_encoder.transform(actual_languages)
correct_predictions = sum(np.array(predicted_labels) == np.array(actual_languages))
accuracy = correct_predictions / len(actual_languages)

print(f"\n✅ Final Test Accuracy: {accuracy:.4f}")


✅ Model loaded successfully!
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 713ms/step

🎯 Model Predictions vs Actual Labels:
Sample    Actual              Predicted           
1         HINDI               HINDI               
2         HINDI               HINDI               
3         HINDI               HINDI               
4         HINDI               HINDI               
5         HINDI               HINDI               
6         HINDI               HINDI               
7         MARWARI             RAJASTHANI          
8         RAJASTHANI          RAJASTHANI          
9         MARWARI             RAJASTHANI          
10        MARWARI             RAJASTHANI          
11        RAJASTHANI          RAJASTHANI          
12        HINDI               HINDI               
13        HINDI               HINDI               
14        HINDI               HINDI               
15        HINDI               HINDI               
16        HINDI               HINDI               
17        HINDI           

MHA+BILSTM+CNN


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Dropout, Input, MultiHeadAttention, LayerNormalization,
    Conv1D, MaxPooling1D, Flatten, LSTM, Bidirectional
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ==============================
# 1️⃣ Load Training Data
# ==============================
csv_file_path ="/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_file_path)
print("✅ Loaded training data!")

# ==============================
# 2️⃣ Preprocess Data
# ==============================
X = df.drop(columns=['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                     'stayYears', 'assertLanguage', 'languagesSpoken', 'state'], errors='ignore')
y = df['assertLanguage']

# Encode labels
language_encoder = LabelEncoder()
y = language_encoder.fit_transform(y)

# Save Label Encoder
joblib.dump(language_encoder, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/language_encoder.pkl")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Save training feature names
train_feature_names = list(X_train.columns)
joblib.dump(train_feature_names,  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/train_feature_names.pkl")

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save Scaler
joblib.dump(scaler,  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/scaler.pkl")

# Reshape for Conv1D and MHA (sequence_length > 1 for CNN)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# One-hot encode labels
y_train_categorical = to_categorical(y_train, num_classes=len(language_encoder.classes_))
y_test_categorical = to_categorical(y_test, num_classes=len(language_encoder.classes_))

print("✅ Preprocessing done!")

# ==============================
# 3️⃣ Define CNN + BiLSTM + MHA Model
# ==============================
input_layer = Input(shape=(X_train.shape[1], 1))  # (timesteps, features)

# 1D Convolution Layer
cnn_output = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)
cnn_output = MaxPooling1D(pool_size=2)(cnn_output)

# Multi-Head Attention Layer
attention_output = MultiHeadAttention(num_heads=4, key_dim=32)(cnn_output, cnn_output)
attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

# BiLSTM Layers for Sequential Learning
bilstm_output = Bidirectional(LSTM(128, return_sequences=True))(attention_output)
bilstm_output = Dropout(0.3)(bilstm_output)
bilstm_output = Bidirectional(LSTM(64, return_sequences=False))(bilstm_output)
bilstm_output = Dropout(0.3)(bilstm_output)

# Fully Connected Layers
x = Dense(64, activation='relu')(bilstm_output)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(len(language_encoder.classes_), activation='softmax')(x)

# Create Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ==============================
# 4️⃣ Train the Model
# ==============================
history = model.fit(
    X_train_reshaped, y_train_categorical,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_categorical),
    callbacks=[early_stopping]
)

# ==============================
# 5️⃣ Save Model
# ==============================
model.save("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final_CNN_BiLSTM_MHA_Model.h5")
print("✅ CNN + BiLSTM + MHA Model training complete and saved!")


✅ Loaded training data!
✅ Preprocessing done!
Epoch 1/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.4334 - loss: 1.9292 - val_accuracy: 0.4569 - val_loss: 1.7079
Epoch 2/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.4596 - loss: 1.7047 - val_accuracy: 0.4942 - val_loss: 1.5903
Epoch 3/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 26s 18ms/step - accuracy: 0.4995 - loss: 1.5900 - val_accuracy: 0.5430 - val_loss: 1.4564
Epoch 4/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 46s 22ms/step - accuracy: 0.5329 - loss: 1.4952 - val_accuracy: 0.5620 - val_loss: 1.3790
Epoch 5/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 37s 19ms/step - accuracy: 0.5574 - loss: 1.4090 - val_accuracy: 0.5916 - val_loss: 1.2783
Epoch 6/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.5847 - loss: 1.3179 - val_accuracy: 0.6081 - val_loss: 1.2210
Epoch 7/100
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.6037 - loss: 1.2461 - val_accuracy: 0.6245 - val_loss: 1.1727
Epoch 8/100
1388/1388 ━

✅ CNN + BiLSTM + MHA Model training complete and saved!


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# ==============================
# 1️⃣ Load Saved Model & Encoders
# ==============================
model_path =  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final_CNN_BiLSTM_MHA_Model.h5"
encoder_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/language_encoder.pkl"
scaler_path =  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/scaler.pkl"
feature_names_path =  "/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/train_feature_names.pkl"

# Load model and preprocessing tools
model = tf.keras.models.load_model(model_path)
language_encoder = joblib.load(encoder_path)
scaler = joblib.load(scaler_path)
train_feature_names = joblib.load(feature_names_path)

print("✅ Model and encoders loaded successfully!")

# ==============================
# 2️⃣ Load and Process Test Data
# ==============================
test_csv_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/test_audio_final.csv"
df_test = pd.read_csv(test_csv_path)

# Extract ground truth
actual_languages = df_test['assertLanguage'].tolist()

# Drop unused columns
drop_columns = ['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                'stayYears', 'assertLanguage', 'languagesSpoken', 'state']
df_test = df_test.drop(columns=[col for col in drop_columns if col in df_test.columns])

# Ensure matching features
for feature in train_feature_names:
    if feature not in df_test.columns:
        df_test[feature] = 0

# Match order
df_test = df_test[train_feature_names]

# ==============================
# 3️⃣ Scale & Reshape for CNN + BiLSTM + MHA
# ==============================
X_test_scaled = scaler.transform(df_test)

# ✅ Reshape: (batch, time_steps=features, channels=1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# ==============================
# 4️⃣ Predict
# ==============================
predictions = model.predict(X_test_reshaped)
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_labels = language_encoder.inverse_transform(predicted_class_indices)

# ==============================
# 5️⃣ Display Results
# ==============================
print("\n🎯 Model Predictions vs Actual Labels:")
print(f"{'Sample':<10}{'Actual':<20}{'Predicted':<20}")

for i, (actual, predicted) in enumerate(zip(actual_languages, predicted_labels)):
    print(f"{i+1:<10}{actual:<20}{predicted:<20}")

# ==============================
# 6️⃣ Accuracy
# ==============================
correct_predictions = sum(np.array(predicted_labels) == np.array(actual_languages))
accuracy = correct_predictions / len(actual_languages)
print(f"\n✅ Final Test Accuracy: {accuracy:.4f}")


✅ Model and encoders loaded successfully!
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step

🎯 Model Predictions vs Actual Labels:
Sample    Actual              Predicted           
1         HINDI               HINDI               
2         HINDI               HINDI               
3         HINDI               HINDI               
4         HINDI               HINDI               
5         HINDI               HINDI               
6         HINDI               HINDI               
7         MARWARI             RAJASTHANI          
8         RAJASTHANI          RAJASTHANI          
9         MARWARI             RAJASTHANI          
10        MARWARI             RAJASTHANI          
11        RAJASTHANI          RAJASTHANI          
12        HINDI               HINDI               
13        HINDI               HINDI               
14        HINDI               HINDI               
15        HINDI               HINDI               
16        HINDI               HINDI               
17        HIN

Ensembeled Model


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ========================
# 1️⃣ Load the CSV
# ========================
csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_file_path)
print("✅ Loaded Data!")

# ========================
# 2️⃣ Preprocessing
# ========================
X = df.drop(columns=['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                     'stayYears', 'assertLanguage', 'languagesSpoken', 'state'], errors='ignore')
y = df['assertLanguage']

# Label encoding
language_encoder = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/language_encoder.pkl")
y_encoded = language_encoder.transform(y)

# Load scaler
scaler = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/scaler.pkl")
X_scaled = scaler.transform(X)

# Reshape for each model
X_mha = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])               # MHA & MHA+BiLSTM
X_cnn = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)              # CNN+BiLSTM+MHA

# ========================
# 3️⃣ Load Models
# ========================
model_mha = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/final colab.h5")
model_bilstm_mha = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final code.h5")
model_cnn_bilstm_mha = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final_CNN_BiLSTM_MHA_Model.h5")

print("✅ Models loaded!")

# ========================
# 4️⃣ Get Predictions
# ========================
pred_mha = model_mha.predict(X_mha)
pred_bilstm_mha = model_bilstm_mha.predict(X_mha)
pred_cnn_bilstm_mha = model_cnn_bilstm_mha.predict(X_cnn)

# Concatenate predictions for meta model
ensemble_input = np.concatenate([pred_mha, pred_bilstm_mha, pred_cnn_bilstm_mha], axis=1)

# ========================
# 5️⃣ Train RandomForest on these predictions
# ========================
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)
meta_model.fit(ensemble_input, y_encoded)
ensemble_preds = meta_model.predict(ensemble_input)

# ========================
# 6️⃣ Evaluation
# ========================
print("✅ Ensemble Model Performance:")
print(classification_report(y_encoded, ensemble_preds, target_names=language_encoder.classes_))
print("🎯 Accuracy:", accuracy_score(y_encoded, ensemble_preds))


✅ Loaded Data!


✅ Models loaded!
2312/2312 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
2312/2312 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step
2312/2312 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step
✅ Ensemble Model Performance:
              precision    recall  f1-score   support

     BENGALI       1.00      1.00      1.00      1510
      BHATRI       1.00      1.00      1.00        88
CHATTISGARHI       1.00      1.00      1.00      1788
       GONDI       1.00      1.00      1.00         2
       HALBI       1.00      1.00      1.00      1163
       HINDI       1.00      1.00      1.00     32397
     KANNADA       1.00      1.00      1.00      6066
     KHORTHA       1.00      1.00      1.00      1442
     KONKANI       1.00      1.00      1.00      1263
    MAITHILI       1.00      1.00      1.00      1289
     MARATHI       1.00      1.00      1.00      4194
     MARWARI       1.00      1.00      1.00      2614
  RAJASTHANI       1.00      1.00      1.00      9872
      TELUGU       1.00      1.00      1.00      9956
        URDU 

In [ ]:
import joblib
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
import pandas as pd

# ============================================
# 1️⃣ Load and Preprocess Full Dataset
# ============================================
csv_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_path)

# Drop metadata columns, keep features and label
X = df.drop(columns=['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                     'stayYears', 'assertLanguage', 'languagesSpoken', 'state'], errors='ignore')
y = df['assertLanguage']

# Label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
joblib.dump(label_encoder, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/label_encoder.pkl")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.4, random_state=42, stratify=y_encoded
)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/scaler.pkl")

# Save X_test and y_test for ensemble use
joblib.dump(X_test_scaled, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/X_test_scaled.pkl")
joblib.dump(y_test, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/y_test.pkl")

# ============================================
# 2️⃣ Load Trained Deep Learning Models
# ============================================
model_mha = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/final colab.h5")
model_bilstm = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final code.h5")
model_cnn = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final_CNN_BiLSTM_MHA_Model.h5")

# ============================================
# 3️⃣ Prepare Input Shapes for Each Model
# ============================================
X_mha = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])
X_bilstm = X_mha
X_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# ============================================
# 4️⃣ Generate Predictions from Each Model
# ============================================
pred_mha = model_mha.predict(X_mha)
pred_bilstm = model_bilstm.predict(X_bilstm)
pred_cnn = model_cnn.predict(X_cnn)

# ============================================
# 5️⃣ Train Random Forest Ensemble
# ============================================
ensemble_input = np.concatenate([pred_mha, pred_bilstm, pred_cnn], axis=1)
rf_ensemble = RandomForestClassifier(n_estimators=100, random_state=42)
rf_ensemble.fit(ensemble_input, y_test)

# ============================================
# 6️⃣ Save Ensemble Model
# ============================================
joblib.dump(rf_ensemble, "/content/drive/MyDrive/IE3 Lab/Sakshi PC/ensemble_language_predictor.pkl")

print("✅ Ensemble model trained and saved successfully!")


925/925 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
925/925 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
925/925 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
✅ Ensemble model trained and saved successfully!


In [ ]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# =====================================
# 1️⃣ Load kabira.csv (only features)
# =====================================
kabira_df = pd.read_csv("/content/drive/MyDrive/IE3 Lab/Sakshi PC/skn csv features.csv")

# Rename columns to match original features (if needed)
kabira_df.columns = [f"feature_{i}" for i in range(kabira_df.shape[1])]

# =====================================
# 2️⃣ Load Preprocessing Objects & Models
# =====================================
scaler = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/scaler.pkl")
label_encoder = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/label_encoder.pkl")
ensemble_model = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/ensemble_language_predictor.pkl")

model_mha = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/final colab.h5")
model_bilstm = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final code.h5")
model_cnn = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final_CNN_BiLSTM_MHA_Model.h5")

# =====================================
# 3️⃣ Scale and Reshape Input
# =====================================
X_kabira_scaled = scaler.transform(kabira_df)

X_kabira_mha = X_kabira_scaled.reshape(X_kabira_scaled.shape[0], 1, X_kabira_scaled.shape[1])
X_kabira_bilstm = X_kabira_mha
X_kabira_cnn = X_kabira_scaled.reshape(X_kabira_scaled.shape[0], X_kabira_scaled.shape[1], 1)

# =====================================
# 4️⃣ Get Predictions from All Models
# =====================================
pred_mha = model_mha.predict(X_kabira_mha)
pred_bilstm = model_bilstm.predict(X_kabira_bilstm)
pred_cnn = model_cnn.predict(X_kabira_cnn)

# =====================================
# 5️⃣ Ensemble Prediction
# =====================================
ensemble_input = np.concatenate([pred_mha, pred_bilstm, pred_cnn], axis=1)
final_preds = ensemble_model.predict(ensemble_input)
final_languages = label_encoder.inverse_transform(final_preds)

# =====================================
# 6️⃣ Output Results
# =====================================
for i, lang in enumerate(final_languages):
    print(f"🔤 Audio Sample {i+1}: Predicted Language → {lang}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 823ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step
🔤 Audio Sample 1: Predicted Language → HINDI
🔤 Audio Sample 2: Predicted Language → TELUGU
🔤 Audio Sample 3: Predicted Language → HINDI


In [ ]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# =====================================
# 1️⃣ Load kabira.csv (only features)
# =====================================
kabira_df = pd.read_csv("/content/drive/MyDrive/IE3 Lab/Sakshi PC/kabira.csv")

# Rename columns to match original features (if needed)
kabira_df.columns = [f"feature_{i}" for i in range(kabira_df.shape[1])]

# =====================================
# 2️⃣ Load Preprocessing Objects & Models
# =====================================
scaler = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/scaler.pkl")
label_encoder = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/label_encoder.pkl")
ensemble_model = joblib.load("/content/drive/MyDrive/IE3 Lab/Sakshi PC/ensemble_language_predictor.pkl")

model_mha = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/Vaani MHA/final colab.h5")
model_bilstm = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final code.h5")
model_cnn = load_model("/content/drive/MyDrive/IE3 Lab/Sakshi PC/VAANI BILSTM+MHA/Final_CNN_BiLSTM_MHA_Model.h5")

# =====================================
# 3️⃣ Scale and Reshape Input
# =====================================
X_kabira_scaled = scaler.transform(kabira_df)

X_kabira_mha = X_kabira_scaled.reshape(X_kabira_scaled.shape[0], 1, X_kabira_scaled.shape[1])
X_kabira_bilstm = X_kabira_mha
X_kabira_cnn = X_kabira_scaled.reshape(X_kabira_scaled.shape[0], X_kabira_scaled.shape[1], 1)

# =====================================
# 4️⃣ Get Predictions from All Models
# =====================================
pred_mha = model_mha.predict(X_kabira_mha)
pred_bilstm = model_bilstm.predict(X_kabira_bilstm)
pred_cnn = model_cnn.predict(X_kabira_cnn)

# =====================================
# 5️⃣ Ensemble Prediction
# =====================================
ensemble_input = np.concatenate([pred_mha, pred_bilstm, pred_cnn], axis=1)
final_preds = ensemble_model.predict(ensemble_input)
final_languages = label_encoder.inverse_transform(final_preds)

# =====================================
# 6️⃣ Output Results
# =====================================
for i, lang in enumerate(final_languages):
    print(f"🔤 Audio Sample {i+1}: Predicted Language → {lang}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step
🔤 Audio Sample 1: Predicted Language → RAJASTHANI
🔤 Audio Sample 2: Predicted Language → MARWARI
🔤 Audio Sample 3: Predicted Language → BENGALI
🔤 Audio Sample 4: Predicted Language → KANNADA
🔤 Audio Sample 5: Predicted Language → URDU
🔤 Audio Sample 6: Predicted Language → HINDI
🔤 Audio Sample 7: Predicted Language → HALBI
🔤 Audio Sample 8: Predicted Language → MARATHI
🔤 Audio Sample 9: Predicted Language → KHORTHA


Sakshi ends here

In [ ]:
# prompt: mount google

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Conv1D, LSTM, Bidirectional, MultiHeadAttention, Flatten, LayerNormalization, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ==============================
# 1️⃣ Load Training Data
# ==============================
csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_file_path)
print("✅ Loaded training data!")

✅ Loaded training data!


In [ ]:
df.columns


Index(['id', 'filename', 'file_url', 'state', 'gender', 'pincode', 'district',
       'stayYears', 'assertLanguage', 'languagesSpoken', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37'],
      dtype='object')

In [ ]:
sahitya=pd.read_csv("/content/drive/MyDrive/new test audios/Sahitya's Audio Features.csv")
nidhi=pd.read_csv("/content/drive/MyDrive/new test audios/Nidhi's Audio Features.csv")
kimaya=pd.read_csv("/content/drive/MyDrive/new test audios/Kimaya's Audio Features.csv")

In [ ]:
sahitya.columns = [f"feature_{i}" for i in range(len(sahitya.columns))]
nidhi.columns = [f"feature_{i}" for i in range(len(nidhi.columns))]
kimaya.columns = [f"feature_{i}" for i in range(len(kimaya.columns))]


In [ ]:
sahitya['assertLanguage'] = 'TELUGU'
nidhi['assertLanguage'] = 'MARATHI'
kimaya['assertLanguage'] = 'HINDI'
import pandas as pd

# Concatenating DataFrames
combined_df = pd.concat([sahitya, nidhi, kimaya], ignore_index=True)


In [ ]:
combined_df=combined_df
X_new=combined_df.drop(columns=['assertLanguage'])
y_new=combined_df['assertLanguage']

In [ ]:
combined_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,assertLanguage
0,-301.82130,162.34149,3.773172,29.347368,5.833158,36.223263,-13.055736,7.063834,-0.404653,-3.601685,...,-1.981790,-2.183699,-1.061338,2.378047,1.560722,1.960453,2.480789,4.246184,3.377369,TELUGU
1,-234.52022,143.98537,-16.144390,7.688753,-0.016806,6.717246,-24.379072,2.334164,-18.932920,-5.769550,...,1.955276,-1.191994,-0.626358,2.216004,1.909258,-0.185616,0.284626,3.413281,-1.186310,MARATHI
2,-318.37576,136.73366,20.810110,45.047370,-4.771094,27.556955,-3.196370,15.037026,2.432315,-2.289925,...,0.508572,-4.296451,-1.692242,-2.544844,-0.938026,-3.195067,-0.604838,-0.066298,-2.088836,HINDI


In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Conv1D, LSTM, Bidirectional, MultiHeadAttention, Flatten, LayerNormalization, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ==============================
# 1️⃣ Load Training Data
# ==============================
csv_file_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/merged_data.csv"
df = pd.read_csv(csv_file_path)
print("✅ Loaded training data!")

# ==============================
# 2️⃣ Preprocess Data
# ==============================
X = df.drop(columns=['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                     'stayYears', 'assertLanguage', 'languagesSpoken', 'state'], errors='ignore')
y = df['assertLanguage']


✅ Loaded training data!


In [ ]:
X_new

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37
0,-301.82130,162.34149,3.773172,29.347368,5.833158,36.223263,-13.055736,7.063834,-0.404653,-3.601685,...,-0.477028,-1.981790,-2.183699,-1.061338,2.378047,1.560722,1.960453,2.480789,4.246184,3.377369
1,-234.52022,143.98537,-16.144390,7.688753,-0.016806,6.717246,-24.379072,2.334164,-18.932920,-5.769550,...,-4.614813,1.955276,-1.191994,-0.626358,2.216004,1.909258,-0.185616,0.284626,3.413281,-1.186310
2,-318.37576,136.73366,20.810110,45.047370,-4.771094,27.556955,-3.196370,15.037026,2.432315,-2.289925,...,-2.471562,0.508572,-4.296451,-1.692242,-2.544844,-0.938026,-3.195067,-0.604838,-0.066298,-2.088836


In [ ]:
X=pd.concat([X, X_new], ignore_index=True)
y=pd.concat([y, y_new], ignore_index=True)

In [ ]:
y

,assertLanguage
0,HINDI
1,HINDI
2,HINDI
3,HINDI
4,HINDI
...,...
73982,RAJASTHANI
73983,RAJASTHANI
73984,TELUGU
73985,MARATHI


In [ ]:
X

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37
0,-297.546265,120.581055,20.237211,59.388645,-0.672819,-14.173330,-29.306034,-17.202627,-15.284561,-21.358829,...,20.510233,24.213826,29.381877,14.743478,0.033548,0.011172,-0.025681,-0.073708,-0.039746,0.011990
1,-260.200684,105.669434,8.691262,58.377419,-17.515112,4.168218,-24.531080,-25.229774,-23.366369,-30.466015,...,21.712901,22.469807,29.498849,17.961922,0.068505,-0.003371,-0.059312,-0.125556,-0.026395,-0.015464
2,-234.857452,80.267685,2.636047,9.870395,-13.524202,-16.229780,-23.384377,-12.242881,-20.520033,-13.439591,...,18.724071,21.145171,23.260481,31.949563,0.014118,-0.005542,0.056249,-0.050925,0.005657,0.001793
3,-288.347992,136.035095,5.828763,57.002117,-23.706793,0.961969,-14.126427,-52.532833,-8.909392,-9.687088,...,21.278267,25.709413,35.111273,14.684528,0.025488,0.024609,0.015288,-0.115441,-0.032343,0.001734
4,-210.954956,106.860130,6.268232,9.759131,-21.126705,-12.630594,-23.158993,-16.264835,-22.510866,-14.486049,...,22.391488,22.408556,23.591935,26.764527,0.068178,0.010451,0.221548,-0.095666,0.060192,-0.007815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73982,-123.170074,79.047691,8.513207,70.706596,-10.343614,-10.557887,-24.143688,-7.584201,-17.439257,-6.821399,...,17.427161,20.728594,20.213408,28.891565,-0.042445,0.040650,-0.133627,0.140569,0.049548,0.012495
73983,-264.190826,116.151489,8.716024,31.138224,-12.527115,-1.554535,-10.416448,-9.566309,-14.856421,-13.804207,...,19.511640,19.491819,19.868303,20.756192,-0.023760,0.033770,0.003179,-0.099469,-0.000492,-0.029672
73984,-301.821300,162.341490,3.773172,29.347368,5.833158,36.223263,-13.055736,7.063834,-0.404653,-3.601685,...,-0.477028,-1.981790,-2.183699,-1.061338,2.378047,1.560722,1.960453,2.480789,4.246184,3.377369
73985,-234.520220,143.985370,-16.144390,7.688753,-0.016806,6.717246,-24.379072,2.334164,-18.932920,-5.769550,...,-4.614813,1.955276,-1.191994,-0.626358,2.216004,1.909258,-0.185616,0.284626,3.413281,-1.186310


# mha+bIlstm+CNN

In [ ]:

# Encode labels
language_encoder = LabelEncoder()
y = language_encoder.fit_transform(y)

# Save Label Encoder
#joblib.dump(language_encoder, r"C:\Users\NMIMS.Student\Desktop\vaani\language_encoder.pkl")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Save training feature names
train_feature_names = list(X_train.columns)
#joblib.dump(train_feature_names, r"C:\Users\NMIMS.Student\Desktop\vaani\train_feature_names.pkl")

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save Scaler
#joblib.dump(scaler, r"C:\Users\NMIMS.Student\Desktop\vaani\scaler.pkl")

# Reshape for CNN + BiLSTM + Attention
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# One-hot encode labels
y_train_categorical = to_categorical(y_train, num_classes=len(language_encoder.classes_))
y_test_categorical = to_categorical(y_test, num_classes=len(language_encoder.classes_))

print("✅ Preprocessing done!")

# ==============================
# 3️⃣ Define Hybrid Model: CNN + BiLSTM + Multi-Head Attention
# ==============================
input_layer = Input(shape=(X_train.shape[1], 1))

# CNN Feature Extraction
cnn = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(input_layer)
cnn = BatchNormalization()(cnn)

# BiLSTM Layer
bilstm = Bidirectional(LSTM(64, return_sequences=True, dropout=0.3))(cnn)

# Multi-Head Attention
attention_output = MultiHeadAttention(num_heads=4, key_dim=32)(bilstm, bilstm)
attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

# Flatten and Fully Connected Layers
x = Flatten()(attention_output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(language_encoder.classes_), activation='softmax')(x)

# Create Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# ==============================
# 4️⃣ Train the Model
# ==============================
history = model.fit(
    X_train_reshaped, y_train_categorical,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test_categorical),
    callbacks=[early_stopping]
)

# ==============================
# 5️⃣ Save Model
# ==============================
model.save("/content/drive/MyDrive/my_trained_hybrid_model_MHA_BILSTM_CNN.h5")
print("✅ Model training complete and saved!")


✅ Preprocessing done!


NameError: name 'Conv1D' is not defined

In [ ]:
joblib.dump(language_encoder, "/content/drive/MyDrive/language_encoder_mha_bilstm_cnn.pkl")
joblib.dump(train_feature_names, "/content/drive/MyDrive/train_feature_names_mha_bilstm_cnn.pkl")
joblib.dump(scaler, "/content/drive/MyDrive/scaler_mha_bilstm_cnn.pkl")


['/content/drive/MyDrive/scaler_mha_bilstm_cnn.pkl']

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# ==============================
# 1️⃣ Load Saved Model & Encoders
# ==============================
model_path = "/content/drive/MyDrive/my_trained_hybrid_model_MHA_BILSTM_CNN.h5"
encoder_path = "/content/drive/MyDrive/language_encoder_mha_bilstm_cnn.pkl"
scaler_path = "/content/drive/MyDrive/scaler_mha_bilstm_cnn.pkl"
feature_names_path = "/content/drive/MyDrive/train_feature_names_mha_bilstm_cnn.pkl"

# Load Model
model = tf.keras.models.load_model(model_path)
print("✅ Model loaded successfully!")

# Load Label Encoder
if os.path.exists(encoder_path):
    language_encoder = joblib.load(encoder_path)
else:
    raise FileNotFoundError("🚨 Label Encoder file not found. Run training script first.")

# Load Scaler
if os.path.exists(scaler_path):
    scaler = joblib.load(scaler_path)
else:
    raise FileNotFoundError("🚨 Scaler file not found. Run training script first.")

# Load Training Feature Names
if os.path.exists(feature_names_path):
    train_feature_names = joblib.load(feature_names_path)
else:
    raise FileNotFoundError("🚨 Training feature names file not found. Run training script first.")

# ==============================
# 2️⃣ Load and Process Test Data
# ==============================
test_csv_path = "/content/drive/MyDrive/IE3 Lab/Sakshi PC/test_audio_final.csv"
df_test = pd.read_csv(test_csv_path)

# Extract actual labels before dropping columns
actual_languages = df_test['assertLanguage'].tolist()

# Drop unnecessary columns
drop_columns = ['id', 'filename', 'file_url', 'gender', 'pincode', 'district',
                'stayYears', 'assertLanguage', 'languagesSpoken', 'state']
df_test = df_test.drop(columns=[col for col in drop_columns if col in df_test.columns])

# Ensure test data has same features as training
for feature in train_feature_names:
    if feature not in df_test.columns:
        df_test[feature] = 0

# Reorder columns to match training
df_test = df_test[train_feature_names]

# ==============================
# 3️⃣ Scale and Reshape Test Data
# ==============================
X_test_new = scaler.transform(df_test)
X_test_reshaped_new = X_test_new.reshape(X_test_new.shape[0], X_test_new.shape[1], 1)

# ==============================
# 4️⃣ Make Predictions
# ==============================
predictions = model.predict(X_test_reshaped_new)
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_labels = language_encoder.inverse_transform(predicted_class_indices)

# ==============================
# 5️⃣ Print Predictions vs Actual Labels
# ==============================
print("\n🎯 Model Predictions vs Actual Labels:")
print(f"{'Sample':<10}{'Actual':<20}{'Predicted':<20}")

for i, (actual, predicted) in enumerate(zip(actual_languages, predicted_labels)):
    print(f"{i+1:<10}{actual:<20}{predicted:<20}")

# ==============================
# 6️⃣ Compute Accuracy
# ==============================
y_test_encoded = language_encoder.transform(actual_languages)
correct_predictions = sum(np.array(predicted_labels) == np.array(actual_languages))
accuracy = correct_predictions / len(actual_languages)

print(f"\n✅ Final Test Accuracy: {accuracy:.4f}")


✅ Model loaded successfully!
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 803ms/step

🎯 Model Predictions vs Actual Labels:
Sample    Actual              Predicted           
1         HINDI               HINDI               
2         HINDI               HINDI               
3         HINDI               HINDI               
4         HINDI               HINDI               
5         HINDI               HINDI               
6         HINDI               MARATHI             
7         MARWARI             RAJASTHANI          
8         RAJASTHANI          RAJASTHANI          
9         MARWARI             RAJASTHANI          
10        MARWARI             RAJASTHANI          
11        RAJASTHANI          RAJASTHANI          
12        HINDI               HINDI               
13        HINDI               HINDI               
14        HINDI               HINDI               
15        HINDI               HINDI               
16        HINDI               HINDI               
17        HINDI           

In [ ]:
# prompt: mount google drive import gooogle

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
joblib.dump(language_encoder, "/content/drive/MyDrive/language_encoder_mha_bilstm_cnn.pkl")
joblib.dump(train_feature_names, "/content/drive/MyDrive/train_feature_names_mha_bilstm_cnn.pkl")
joblib.dump(scaler, "/content/drive/MyDrive/scaler_mha_bilstm_cnn.pkl")

NameError: name 'language_encoder' is not defined

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Load the Keras model
model = load_model("/content/drive/MyDrive/my_trained_hybrid_model_MHA_BILSTM_CNN.h5", compile=False)
language_encoder = joblib.load("/content/drive/MyDrive/language_encoder_mha_bilstm_cnn.pkl")
train_feature_names = joblib.load("/content/drive/MyDrive/train_feature_names_mha_bilstm_cnn.pkl")
scaler = joblib.load("/content/drive/MyDrive/scaler_mha_bilstm_cnn.pkl")

sahitya=pd.read_csv("/content/drive/MyDrive/new test audios/Kimaya's Audio Features.csv")
sahitya.columns = [f"feature_{i}" for i in range(len(sahitya.columns))]
sahitya_new = scaler.transform(sahitya)
sahitya_reshaped_new = sahitya_new.reshape(sahitya_new.shape[0], sahitya_new.shape[1], 1)
predictions = model.predict(sahitya_reshaped_new)
predictions
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_class_indices
predicted_labels = language_encoder.inverse_transform(predicted_class_indices)
predicted_labels

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step


array(['HINDI'], dtype=object)

In [ ]:
sahitya.columns = [f"feature_{i}" for i in range(len(sahitya.columns))]
sahitya_new = scaler.transform(sahitya)
sahitya_reshaped_new = sahitya_new.reshape(sahitya_new.shape[0], sahitya_new.shape[1], 1)

In [ ]:
predictions = model.predict(sahitya_reshaped_new)
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_labels = language_encoder.inverse_transform(predicted_class_indices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [ ]:
predicted_labels

array(['MARATHI'], dtype=object)

In [ ]:
#STREAMLIT
import streamlit as st
import numpy as np
import pandas as pd
import librosa
import os
import joblib
import requests
from googletrans import Translator
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import google.generativeai as genai
from PyPDF2.errors import PdfReadError
import tempfile
from dotenv import load_dotenv
from tensorflow.keras.models import load_model

# Load environment variables
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

if not os.getenv("GOOGLE_API_KEY"):
    st.error("Google API key not loaded!")

# Initialize translator
translator = Translator()

# Configuration
LANGUAGE_DICT = {'HINDI': 'hi-IN', 'MARATHI': 'mr-IN'}
SARVAM_API_KEY = "ff014cc1-c60d-4304-b097-76e135ab6ac1"

# ========== CORE FUNCTIONS ==========

def get_pdf_text(pdf_docs):
    """Extract text from uploaded PDF files"""
    text = ""
    for pdf in pdf_docs:
        try:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        except Exception as e:
            st.warning(f"Couldn't read {pdf.name}: {str(e)}")
    return text

def get_text_chunks(text):
    """Split text into chunks for processing"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=10000,
        chunk_overlap=1000
    )
    return text_splitter.split_text(text)

def get_vector_store(text_chunks):
    """Create and save vector store"""
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain():
    """Create QA chain with prompt template"""
    prompt_template = """
    Answer the question from the context. If answer isn't in context,
    say "answer not available in context". Don't make up answers.

    Context:\n{context}\n
    Question:\n{question}\n
    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template,
                          input_variables=["context", "question"])
    return load_qa_chain(model, chain_type="stuff", prompt=prompt)

def get_answer(question):
    """Get answer from vector store"""
    try:
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        new_db = FAISS.load_local("faiss_index", embeddings,
                                 allow_dangerous_deserialization=True)
        docs = new_db.similarity_search(question)
        chain = get_conversational_chain()
        response = chain({"input_documents": docs, "question": question},
                        return_only_outputs=True)
        return response["output_text"]
    except Exception as e:
        return f"Error: {str(e)}"

def extract_features(audio_file):
    """Extract exactly 38 MFCC features to match the scaler's expectations"""
    y, sr = librosa.load(audio_file, sr=None)

    # Extract only MFCCs with 38 coefficients
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=38)

    # Take mean across time and flatten to 1D array
    features = np.mean(mfccs.T, axis=0)

    return features

def speech_to_text(audio_path, language_code):
    """Convert speech to text using Sarvam API"""
    url = "https://api.sarvam.ai/speech-to-text"
    headers = {'api-subscription-key': SARVAM_API_KEY}
    payload = {
        'model': 'saarika:v1',
        'language_code': LANGUAGE_DICT.get(language_code, 'hi-IN'),
        'with_timesteps': 'false'
    }

    try:
        with open(audio_path, 'rb') as f:
            files = [('file', (os.path.basename(audio_path), f, 'audio/wav'))]
            response = requests.post(url, headers=headers, data=payload, files=files)
            return response.json().get('text', '') if response.status_code == 200 else None
    except Exception as e:
        st.error(f"API error: {str(e)}")
        return None

@st.cache_resource
def load_language_models():
    """Load all language detection models"""
    try:
        return {
            'scaler': joblib.load(r"D:\Nidhi\Audio chatbot\scaler.pkl"),
            'encoder': joblib.load(r"D:\Nidhi\Audio chatbot\label_encoder.pkl"),
            'ensemble': joblib.load(r"D:\Nidhi\Audio chatbot\ensemble_language_predictor.pkl"),
            'models': {
                'mha': load_model(r"D:\Nidhi\Audio chatbot\MHA.h5"),
                'bilstm': load_model(r"D:\Nidhi\Audio chatbot\MHA Bilstm.h5"),
                'cnn': load_model(r"D:\Nidhi\Audio chatbot\Final_CNN_BiLSTM_MHA_Model.h5")
            }
        }
    except Exception as e:
        st.error(f"Model loading failed: {str(e)}")
        return None
def detect_language(features, models_dict):
    try:
        # Ensure we have exactly 38 features
        if len(features) != 38:
            raise ValueError(f"Expected 38 features, got {len(features)}")

        # Reshape for scaler (n_samples=1, n_features=38)
        features = np.array(features).reshape(1, -1)

        # Scale features
        features_scaled = models_dict['scaler'].transform(features)

        # Prepare inputs for each model
        X_mha = features_scaled.reshape(1, 1, 38)  # (batch, timesteps, features)
        X_cnn = features_scaled.reshape(1, 38, 1)   # (batch, features, channels)

        # Get predictions
        pred_mha = models_dict['models']['mha'].predict(X_mha, verbose=0)
        pred_bilstm = models_dict['models']['bilstm'].predict(X_mha, verbose=0)
        pred_cnn = models_dict['models']['cnn'].predict(X_cnn, verbose=0)

        # Combine predictions
        ensemble_input = np.concatenate([pred_mha, pred_bilstm, pred_cnn], axis=1)
        final_pred = models_dict['ensemble'].predict(ensemble_input)

        return models_dict['encoder'].inverse_transform(final_pred)[0]

    except Exception as e:
        st.error(f"Detection Error: {str(e)}")
        st.error(f"Input shape: {features.shape if 'features' in locals() else 'N/A'}")
        return None
# ========== STREAMLIT APP ==========

def main():
    st.set_page_config("Multilingual Audio Q&A System", layout="wide")
    st.title("🎙️ Multilingual Audio Q&A System")

    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    models_dict = load_language_models()

    # Sidebar
    with st.sidebar:
        st.header("📂 Knowledge Base")
        uploaded_files = st.file_uploader("Upload PDFs",
                                        accept_multiple_files=True,
                                        type=["pdf"])
        if uploaded_files:
            with st.spinner("Processing..."):
                raw_text = get_pdf_text(uploaded_files)
                if raw_text:
                    get_vector_store(get_text_chunks(raw_text))
                    st.success("Knowledge base ready!")

    # Main content
    col1, col2 = st.columns([1, 2])

    with col1:
        st.header("🎤 Ask Question")
        audio_file = st.file_uploader("Upload WAV", type=["wav"])

        if audio_file and models_dict:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
                tmp.write(audio_file.read())
                audio_path = tmp.name

            with st.spinner("Analyzing..."):
                try:
                    features = extract_features(audio_path)
                    language = detect_language(features, models_dict)
                    if language:
                        st.success(f"Language: {language}")
                        if question := speech_to_text(audio_path, language):
                            st.session_state.chat_history.append({
                                "role": "user",
                                "content": question,
                                "language": language
                            })
                finally:
                    os.unlink(audio_path)

    with col2:
        st.header("💬 Conversation")
        for msg in st.session_state.chat_history:
            with st.chat_message(msg["role"]):
                st.write(msg["content"])

        if st.session_state.chat_history and st.session_state.chat_history[-1]["role"] == "user":
            last = st.session_state.chat_history[-1]
            answer = get_answer(last["content"])

            if last["language"] != "ENGLISH":
                try:
                    answer = translator.translate(answer, src='en', dest=last["language"].lower()).text
                except:
                    answer += " (translation failed)"

            st.session_state.chat_history.append({
                "role": "assistant",
                "content": answer,
                "language": last["language"]
            })
            st.rerun()

if __name__ == "__main__":
    main()